In [29]:
import os
import time
import torch

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from models import ShallowNN
from utils import load_file, get_all_possible_pairs
from evals import evaluate, pairwise_euclidean_distance , influence, calculate_hessian_flattened
from evals import euclidean_distance, manhattan_distance, pairwise_euclidean_distance, accumulated_proximity, full_accumulated_proximity

features = 197
batch_size = 64
loss_fn = torch.nn.L1Loss()

In [6]:
client_ids = [f"{i}_{j}" for i in range(4) for j in range(6)]

## 500 Global Round and 1 Local Round

In [3]:
global_model = ShallowNN(features)
global_model.load_state_dict(torch.load('checkpt/epoch_500/_fedl_global_500.pth'))

<All keys matched successfully>

In [8]:
def hessian(client):
    
    dataset = torch.load("trainpt/"+ str(client)+".pt")
    data_loader = DataLoader(dataset, batch_size, shuffle=True)
                                              
    fed_hessian_mat , _  = calculate_hessian_flattened(global_model,loss_fn,data_loader)
    
    return fed_hessian_mat

In [9]:
for client in client_ids:
    fed_hessian_mat = hessian(client)
    torch.save(fed_hessian_mat, "hessians/epoch_500/train_fed/"+str(client)+".pth")
    print("Client " +str(client)+ " done.")

Calculation time of Gradients 0.017606019973754883
Calculation time of Hessian 9.303941015402476
Client 0_0 done.
Calculation time of Gradients 0.01801896095275879
Calculation time of Hessian 9.503507733345032
Client 0_1 done.
Calculation time of Gradients 0.018792152404785156
Calculation time of Hessian 9.581230354309081
Client 0_2 done.
Calculation time of Gradients 0.017399072647094727
Calculation time of Hessian 9.333001538117726
Client 0_3 done.
Calculation time of Gradients 0.01732492446899414
Calculation time of Hessian 9.43962384859721
Client 0_4 done.
Calculation time of Gradients 0.017280101776123047
Calculation time of Hessian 9.395591231187185
Client 0_5 done.
Calculation time of Gradients 0.018857955932617188
Calculation time of Hessian 10.077910633881887
Client 1_0 done.
Calculation time of Gradients 0.019461870193481445
Calculation time of Hessian 10.016091732184092
Client 1_1 done.
Calculation time of Gradients 0.01909804344177246
Calculation time of Hessian 10.48528400

In [ ]:
eigen_values = []
for client in client_ids:
    client_hess = torch.load("hessians/epoch_500/train_fed/"+str(client)+".pth")
    eigenvalues = np.linalg.eigvals(client_hess)
    sorted_eigenvalues = np.sort(eigenvalues)[::-1]
    top_eigenvalues = sorted_eigenvalues[:1]
    eigen_values.append(round(top_eigenvalues[0].real,4))
    print(round(top_eigenvalues[0].real,4))

1.3976
1.3468
1.6332
3.4409
1.4376
1.5074
1.097
1.0757
0.6723
3.4592
0.7504


## 1 Global Round and 25 Local Round

In [2]:
global_model = ShallowNN(features)
global_model.load_state_dict(torch.load('checkpt/epoch_25/1_rounds_25_epochs_per_round/_fedl_global_1_25.pth'))

<All keys matched successfully>

In [7]:
def hessian(client):
    
    dataset = torch.load("trainpt/"+ str(client)+".pt")
    data_loader = DataLoader(dataset, batch_size, shuffle=True)
                                              
    fed_hessian_mat , _  = calculate_hessian_flattened(global_model,loss_fn,data_loader)
    
    return fed_hessian_mat

In [8]:
for client in client_ids:
    fed_hessian_mat = hessian(client)
    torch.save(fed_hessian_mat, "hessians/epoch_25/train_fed/"+str(client)+".pth")
    print("Client " +str(client)+ " done.")

Calculation time of Gradients 0.05275988578796387
Calculation time of Hessian 9.325196115175883
Client 0_0 done.
Calculation time of Gradients 0.018674373626708984
Calculation time of Hessian 9.453354402383168
Client 0_1 done.
Calculation time of Gradients 0.022162914276123047
Calculation time of Hessian 9.664005148410798
Client 0_2 done.
Calculation time of Gradients 0.017602205276489258
Calculation time of Hessian 9.44288760026296
Client 0_3 done.
Calculation time of Gradients 0.01822829246520996
Calculation time of Hessian 9.449368317921957
Client 0_4 done.
Calculation time of Gradients 0.017702817916870117
Calculation time of Hessian 9.337438428401947
Client 0_5 done.
Calculation time of Gradients 0.018970251083374023
Calculation time of Hessian 50.031545066833495
Client 1_0 done.
Calculation time of Gradients 0.019299030303955078
Calculation time of Hessian 10.383516518274943
Client 1_1 done.
Calculation time of Gradients 0.02052783966064453
Calculation time of Hessian 10.36624616

In [30]:
matrix_dict_fed = {
        key: torch.load("hessians/epoch_25/train_fed/"+str(client)+".pth")
        for key in client_ids
    }

In [31]:
eccentricities = []

for client in client_ids:
    client_matrix = torch.load("hessians/epoch_25/train_fed/" + str(client) + ".pth")
    acc_proximity = 0.0
    for key in matrix_dict_fed:
        distance = accumulated_proximity(matrix_dict_fed[key], client_matrix, euclidean_distance)
        acc_proximity += distance
    
    eccentricities.append(round(acc_proximity.item(),4))
    print(round(acc_proximity.item(),4))

12624.96
12513.3584
23060.1074
37059.5508
23537.2656
14262.4463
10858.4521
10763.6084
17709.8262
32710.5
15987.3975
14450.4844
5246.8604
5286.2817
4596.5171
6220.5342
5011.7891
4133.6719
4195.6997
3905.4023
3567.9146
5247.8247
3259.3577
0.0


In [ ]:
full_ecc = []
full_acc_proximity = sum(eccentricities)

for item in eccentricities:
    full_ecc.append(round((item/full_acc_proximity),4))

In [ ]:
ecc_hessian = {"client id": client_ids, "hess_ecc": full_ecc}
ecc_hessian = pd.DataFrame(ecc_hessian)
ecc_hessian.to_csv("insights/eccentricity_with_global_model_based_3_at_25_epoch.csv" , index=False)

### Hessian Matrix for isolated model

In [20]:
def hessian_iso(client):
    
    dataset = torch.load("trainpt/"+ str(client)+".pt")
    data_loader = DataLoader(dataset, batch_size, shuffle=True)
    
    client_model = ShallowNN(features)
    client_model.load_state_dict(torch.load("checkpt/epoch_25/isolated/batch64_client_"+ str(client)+ ".pth"))
                                              
    fed_hessian_mat , _  = calculate_hessian_flattened(client_model,loss_fn,data_loader)
    
    return fed_hessian_mat

In [21]:
for client in client_ids:
    fed_hessian_mat = hessian_iso(client)
    torch.save(fed_hessian_mat, "hessians/epoch_25/iso/"+str(client)+".pth")
    print("Client " +str(client)+ " done.")

Calculation time of Gradients 0.02161884307861328
Calculation time of Hessian 29.918104366461435
Client 0_0 done.
Calculation time of Gradients 0.019407987594604492
Calculation time of Hessian 9.518111634254456
Client 0_1 done.
Calculation time of Gradients 0.017426013946533203
Calculation time of Hessian 9.419407320022582
Client 0_2 done.
Calculation time of Gradients 0.018080949783325195
Calculation time of Hessian 9.39827203353246
Client 0_3 done.
Calculation time of Gradients 0.018204927444458008
Calculation time of Hessian 9.36241496404012
Client 0_4 done.
Calculation time of Gradients 0.017712116241455078
Calculation time of Hessian 9.396783502896627
Client 0_5 done.
Calculation time of Gradients 0.019743919372558594
Calculation time of Hessian 10.352544565995535
Client 1_0 done.
Calculation time of Gradients 0.020287275314331055
Calculation time of Hessian 10.397171564896901
Client 1_1 done.
Calculation time of Gradients 0.01963186264038086
Calculation time of Hessian 10.3426468

In [32]:
matrix_dict_iso = {
        key: torch.load("hessians/epoch_25/iso/"+str(client)+".pth")
        for key in client_ids
    }

In [33]:
eccentricities = []

for client in client_ids:
    client_matrix = torch.load("hessians/epoch_25/iso/" + str(client) + ".pth")
    acc_proximity = 0.0
    for key in matrix_dict_iso:
        distance = accumulated_proximity(matrix_dict_iso[key], client_matrix, euclidean_distance)
        acc_proximity += distance
    
    eccentricities.append(round(acc_proximity.item(),4))
    print(round(acc_proximity.item(),4))

2304.0972
2304.5659
4657.9224
2604.3494
3292.7405
2741.8596
2627.2683
2231.8069
3543.1577
6282.6997
2381.5076
2331.6155
2651.3376
2435.9534
2306.0681
2369.8713
2979.9004
2351.564
3112.4338
3164.4961
3666.1978
2933.1917
3264.3064
0.0


In [34]:
full_ecc = []
full_acc_proximity = sum(eccentricities)

for item in eccentricities:
    full_ecc.append(round((item/full_acc_proximity),4))

In [43]:
full_ecc

[0.0336,
 0.0336,
 0.068,
 0.038,
 0.048,
 0.04,
 0.0383,
 0.0326,
 0.0517,
 0.0917,
 0.0347,
 0.034,
 0.0387,
 0.0355,
 0.0336,
 0.0346,
 0.0435,
 0.0343,
 0.0454,
 0.0462,
 0.0535,
 0.0428,
 0.0476,
 0.0]

In [40]:
ecc_hessian = {"client id": client_ids, "hess_ecc": full_ecc}
ecc_hessian = pd.DataFrame(ecc_hessian)
ecc_hessian.to_csv("insights/eccentricity_with_isolated_model_based_3_at_25_epoch.csv" , index=False)